# Preparations

## Running the notebook

Make sure you can run the notebook:
- if you work on the IT container, follow the instructions to run notebooks remotely. Make sure you have the environment `lst-school-2024-02-mcp` activated
- if you work on PIC, same as above
- if you work locally on your laptop, make sure the environment where you installed `magic-cta-pipe` is active

The runbook can be found in the `magic-cta-pipe-analysis-school-2024` repository. If you did not clone it yet, do it now (from wherever you want to run it, IT, PIC, or locally):

```
git clone https://github.com/cta-observatory/magic-cta-pipe-analysis-school-2024.git
cd magic-cta-pipe-analysis-school-2024
```

If you already have the repository cloned, just to be sure it is updated, do a pull:

```
git pull
```

This notebook should be there, i.e. called `mcp_real_data_first_steps.ipynb`. 

We can now proceed!

## Check environment

Check that the correct version of *magic-cta-pipe* is installed

In [1]:
import magicctapipe

print(magicctapipe.__version__)

0.4.1.dev37+gf962388


For you it should give 0.4.1 (in my case the version is something strange because I am using the development version)

# Joint data table

Do you have joint data? The question is of course important. Usually you will know that there are joint data taken since it is usually reported in LST ELOGs. But the following link is more helpful:

https://docs.google.com/spreadsheets/d/1Tya0tlK-3fuN6_vXOU5FwJruis5vBA9FALAPLFHOfBQ/edit#gid=1066216668

You will find something like the following:

<img src="files/joint_table.png">

Several columns:
- Date: the date of observation in LST convention i.e. the one of the evening when observations started (MAGIC uses the day after)
- Source Name: this comes from MAGIC, but the ones used in LST are very similar
- Conditions: dark/moon/twilight
- Trigger: which trigger type was used in MAGIC, either L3T (standard), SumT (sum trigger), HaST (hardwa stereo trigger), L1_M1, L1_M2 (mono M1 or M2)
- Stereo: if data taken in stereoscopic mode
- HV: high voltage settings applied in MAGIC, either nominal or reduced
- Min Zd, Max Zd: minimum/maximum zenith angle during the LST observation
- LST Data runs: LST-1 data runs with joint data
- MAGIC first/last run: which range of MAGIC runs correspond to the above LST data runs
- Obs time: observation time on the same source (within 5deg)
- Same wobble: observation time when the two systems were pointing at the same target within 0.1deg

There is work ongoing by Ievgen and Volodymyr to have a LST-1 and LST-1+MAGIC database available, hopefully soon! 

Therefore, to know if you have joint data for a given source and night, just search in the table if there is an entry (or use the "query" sheet) for it (NB: date is in LST convention!).

# Where do you find the data?

When you know which data you need, you want to analyze them. You can find the data in the IT container
for MAGIC, only the runs which are tagged as joint according to the previous table are copied over.

- For MAGIC, find the data in: `/fefs/onsite/common/MAGIC/data/[tel_id]/event/Calibrated/YYYY/MM/DD`
   - tel_id can be either M1 or M2
   - YYYY, MM, DD is year, month and day
   - NOTE: day in MAGIC convention is one day after as the one in the LST convention!!! E.g. if LST data are from 2024-02-05, in MAGIC they will end up in the directory 2024/02/06

- For MAGIC we use the so called “calibrated” files (~DL1a level in CTA data levels scheme)
   - one file per subrun (one per telescope), numbers of subruns depending on trigger rate (fixed number of events per subrun); one run = one wobble position (if in wobble mode)
   - they have the substring _Y_ in the name (other levels have a different letter)
   - they are automatically generated by the OnSite Analysis (OSA i.e. the corresponding of LSTOSA of LST) starting from RAW data

- For LST-1, we use DL1 data, you can find files in `/fefs/aswg/data/real/DL1/YYYYMMDD/v0.10/tailcut84/`
   - version tag can change if data are reprocessed
   - cleaning tag can also change (multiple tags with the same version tag)


# First analysis steps

- We start from calibrated files because in the standard MAGIC analysis if in one of the telescope an event does not pass the filters (e.g. cleaning, or others), then the event is removed completely
   - but with the three telescopes, that event may be seen by LST! So we want to keep them

- Therefore, for real data, the first step of the joint analysis requires to:
   - take MAGIC calibrated files, and process them to reach DL1b level
   - cleaning+parameters computation (+pointing interpolation)
   - via the `magic_calib_to_dl1` script

- Then, MAGIC and LST-1 data are at the same data level
   - we can find the (software) coincidence between the events
   - via the `lst1_magic_event_coincidence` script

- After the coincident events are found, one can use the multi-telescope (at least two) information to reconstruct the stereo parameters
   - via the `lst1_magic_stereo_reco` script
   

<img src="files/pipeline_scheme.png">


## Data files for this notebook


The data files needed for this notebook are available on the IT container and at PIC. If you work locally on your laptop, you should have them in some location on your pc.

I selected a very small subsample from the Crab dataset used for the MAGIC+LST-1 performance paper, just to show how to run things. Below you find also subsections where I explain how to "extend" the apporach in a real analysis scenario.

We start with the MAGIC input files.

In [2]:
# location of the MAGIC input files, choose the one it applies to you

#magic_input_file_m1 = "/fefs/aswg/workspace/analysis-school-2024/MCP/dl0_dl1_coincidence_stereo/input/MAGIC/M1/20210214_M1_05094483.008_Y_CrabNebula-W0.40+215.root" # IT container
#magic_input_file_m1 = "/pnfs/pic.es/data/magic/Users/analysis-school-2024/MCP/dl0_dl1_coincidence_stereo/input/MAGIC/M1/20210214_M1_05094483.008_Y_CrabNebula-W0.40+215.root" # PIC
magic_input_file_m1 = "/home/mpp/Software/LST_SW_2024/MCP/dl0_dl1_coincidence_stereo/input/MAGIC/M1/20210214_M1_05094483.008_Y_CrabNebula-W0.40+215.root" # your local pc

#magic_input_file_m2 = "/fefs/aswg/workspace/analysis-school-2024/MCP/dl0_dl1_coincidence_stereo/input/MAGIC/M2/20210214_M2_05094483.008_Y_CrabNebula-W0.40+215.root" # IT container
#magic_input_file_m2 = "/pnfs/pic.es/data/magic/Users/analysis-school-2024/MCP/dl0_dl1_coincidence_stereo/input/MAGIC/M2/20210214_M2_05094483.008_Y_CrabNebula-W0.40+215.root" # PIC
magic_input_file_m2 = "/home/mpp/Software/LST_SW_2024/MCP/dl0_dl1_coincidence_stereo/input/MAGIC/M2/20210214_M2_05094483.008_Y_CrabNebula-W0.40+215.root"

We want now to run the script `magic_calib_to_dl1`. We first check the help:

In [3]:
!magic_calib_to_dl1 -h

usage: magic_calib_to_dl1 [-h] --input-file INPUT_FILE
                          [--output-dir OUTPUT_DIR]
                          [--config-file CONFIG_FILE] [--max-evt MAX_EVENTS]
                          [--process-run]

options:
  -h, --help            show this help message and exit
  --input-file INPUT_FILE, -i INPUT_FILE
                        Path to an input MAGIC calibrated data file
  --output-dir OUTPUT_DIR, -o OUTPUT_DIR
                        Path to a directory where to save an output DL1 data
                        file
  --config-file CONFIG_FILE, -c CONFIG_FILE
                        Path to a configuration file
  --max-evt MAX_EVENTS, -m MAX_EVENTS
                        Max. number of processed showers
  --process-run         Process the events of all the subrun files at once


Most of the options are self-explanatory. The one requiring some explanation is `--process-run`. If this option is passed to the script when we call it, it will process all the subruns belonging to the one we give as input file, if present.

In our case, the directories contain two subruns each. This means that `magic_calib_to_dl1` will process the two subruns together.

Under the hood, `magic_calib_to_dl1` uses the class `MAGICEventSource` implemented in the `ctapipe_io_magic` module (https://github.com/cta-observatory/ctapipe_io_magic). This is the event source that reads MAGIC calibrated files and stores the information for each event in a `ctapipe` compliant data structure (the `ArrayEventContainer`).

We also need a configuration file. There is one provided with the hands-on.

In [4]:
#config_file = "/fefs/aswg/workspace/analysis-school-2024/MCP/dl0_dl1_coincidence_stereo/conf/config.yaml" # IT container
#config_file = "/pnfs/pic.es/data/magic/Users/analysis-school-2024/MCP/dl0_dl1_coincidence_stereo/conf/config.yaml" # PIC
config_file = "/home/mpp/Software/LST_SW_2024/MCP/dl0_dl1_coincidence_stereo/conf/config.yaml"
! cat $config_file

mc_tel_ids:
    LST-1: 1
    LST-2: 0
    LST-3: 0
    LST-4: 0
    MAGIC-I: 2
    MAGIC-II: 3


LST:
    image_extractor:
        type: "LocalPeakWindowSum"
        window_shift: 4
        window_width: 8

    increase_nsb:
        use: true
        extra_noise_in_dim_pixels: 1.27
        extra_bias_in_dim_pixels: 0.665
        transition_charge: 8
        extra_noise_in_bright_pixels: 2.08

    increase_psf:
        use: false
        fraction: null

    tailcuts_clean:
        picture_thresh: 8
        boundary_thresh: 4
        keep_isolated_pixels: false
        min_number_picture_neighbors: 2

    time_delta_cleaning:
        use: true
        min_number_neighbors: 1
        time_limit: 2

    dynamic_cleaning:
        use: true
        threshold: 267
        fraction: 0.03

    use_only_main_island: false


MAGIC:
    image_extractor:
        type: "SlidingWindowMaxSum"
        window_width: 5
        apply_integration_correction: false

    charge_correction:
        use: true


The important parts of the configuration file for this script are:

```
mc_tel_ids:
    LST-1: 1
    LST-2: 0
    LST-3: 0
    LST-4: 0
    MAGIC-I: 2
    MAGIC-II: 3
```

This is needed to assign the telescope IDs for MAGIC and LST-1 (so also in later steps). If you use the standard MC used for joint analysis, the IDs in the default configuration file are the ones you need.

The other important part is the one with the settings for the MAGIC cleaning:

```
MAGIC:
    image_extractor:
        type: "SlidingWindowMaxSum"
        window_width: 5
        apply_integration_correction: false

    charge_correction:
        use: true
        factor: 1.143

    magic_clean:
        use_time: true
        use_sum: true
        picture_thresh: 6
        boundary_thresh: 3.5
        max_time_off: 4.5
        max_time_diff: 1.5
        find_hotpixels: true
        pedestal_type: "from_extractor_rndm"
```

They are settings specific to MAGIC, no need to change them.

We create the output directory.

In [5]:
out_magic_to_dl1 = f"./output/dl0_to_dl1"
!mkdir -p $out_magic_to_dl1

Now we run `magic_calib_to_dl1` for M1 and then M2.

In [6]:
!magic_calib_to_dl1\
--input-file $magic_input_file_m1\
--output-dir $out_magic_to_dl1\
--config-file $config_file\
--process-run


Input file: /home/mpp/Software/LST_SW_2024/MCP/dl0_dl1_coincidence_stereo/input/MAGIC/M1/20210214_M1_05094483.008_Y_CrabNebula-W0.40+215.root

Is simulation: False

Observation ID: 5094483
Telescope ID: 1

Is stereo trigger: True
Is SUM trigger: False

The following files are found to read drive reports:
/home/mpp/Software/LST_SW_2024/MCP/dl0_dl1_coincidence_stereo/input/MAGIC/M1/20210214_M1_05094483.008_Y_CrabNebula-W0.40+215.root
/home/mpp/Software/LST_SW_2024/MCP/dl0_dl1_coincidence_stereo/input/MAGIC/M1/20210214_M1_05094483.009_Y_CrabNebula-W0.40+215.root

Process run: True

MAGIC image cleaning:
    use_time: True
    use_sum: True
    picture_thresh: 6
    boundary_thresh: 3.5
    max_time_off: 4.5
    max_time_diff: 1.5
    find_hotpixels: True
    pedestal_type: from_extractor_rndm 

Processing the events...
Interpolating cosmic events information from 16 drive reports.
Drive reports available from 2021-02-13 21:54:08.035 to 2021-02-13 21:56:14.525.
0 events
--> 2 event (event

--> 545 event (event ID: 102223) could not survive the image cleaning. Skipping...
--> 559 event (event ID: 102239) could not survive the image cleaning. Skipping...
--> 570 event (event ID: 102250) could not survive the image cleaning. Skipping...
--> 586 event (event ID: 102267) could not survive the image cleaning. Skipping...
--> 599 event (event ID: 102281) could not survive the image cleaning. Skipping...
600 events
--> 605 event (event ID: 102287) could not survive the image cleaning. Skipping...
--> 624 event (event ID: 102309) could not survive the image cleaning. Skipping...
--> 625 event (event ID: 102310) could not survive the image cleaning. Skipping...
--> 626 event (event ID: 102311) could not survive the image cleaning. Skipping...
--> 628 event (event ID: 102313) could not survive the image cleaning. Skipping...
--> 632 event (event ID: 102317) could not survive the image cleaning. Skipping...
--> 637 event (event ID: 102322) could not survive the image cleaning. Skipp

--> 1196 event (event ID: 102918) could not survive the image cleaning. Skipping...
--> 1199 event (event ID: 102921) could not survive the image cleaning. Skipping...
1200 events
--> 1205 event (event ID: 102927) could not survive the image cleaning. Skipping...
--> 1213 event (event ID: 102935) could not survive the image cleaning. Skipping...
--> 1215 event (event ID: 102937) could not survive the image cleaning. Skipping...
--> 1216 event (event ID: 102938) could not survive the image cleaning. Skipping...
--> 1227 event (event ID: 102952) could not survive the image cleaning. Skipping...
--> 1235 event (event ID: 102961) could not survive the image cleaning. Skipping...
--> 1250 event (event ID: 102976) could not survive the image cleaning. Skipping...
--> 1251 event (event ID: 102977) could not survive the image cleaning. Skipping...
--> 1259 event (event ID: 102985) could not survive the image cleaning. Skipping...
--> 1269 event (event ID: 102995) could not survive the image cl

--> 1793 event (event ID: 103546) could not survive the image cleaning. Skipping...
--> 1799 event (event ID: 103554) could not survive the image cleaning. Skipping...
1800 events
--> 1815 event (event ID: 103570) could not survive the image cleaning. Skipping...
--> 1820 event (event ID: 103575) could not survive the image cleaning. Skipping...
--> 1829 event (event ID: 103584) could not survive the image cleaning. Skipping...
--> 1838 event (event ID: 103593) could not survive the image cleaning. Skipping...
--> 1843 event (event ID: 103599) could not survive the image cleaning. Skipping...
--> 1854 event (event ID: 103611) could not survive the image cleaning. Skipping...
--> 1864 event (event ID: 103622) could not survive the image cleaning. Skipping...
--> 1871 event (event ID: 103629) could not survive the image cleaning. Skipping...
--> 1879 event (event ID: 103638) could not survive the image cleaning. Skipping...
--> 1881 event (event ID: 103640) could not survive the image cl

--> 2410 event (event ID: 104201) could not survive the image cleaning. Skipping...
--> 2413 event (event ID: 104204) could not survive the image cleaning. Skipping...
--> 2425 event (event ID: 104217) could not survive the image cleaning. Skipping...
--> 2428 event (event ID: 104221) could not survive the image cleaning. Skipping...
--> 2440 event (event ID: 104233) could not survive the image cleaning. Skipping...
--> 2441 event (event ID: 104234) could not survive the image cleaning. Skipping...
--> 2450 event (event ID: 104245) could not survive the image cleaning. Skipping...
--> 2451 event (event ID: 104246) could not survive the image cleaning. Skipping...
--> 2460 event (event ID: 104256) could not survive the image cleaning. Skipping...
--> 2465 event (event ID: 104261) could not survive the image cleaning. Skipping...
--> 2468 event (event ID: 104264) could not survive the image cleaning. Skipping...
--> 2483 event (event ID: 104281) could not survive the image cleaning. Skip

...

--> 23790 event (event ID: 127652) could not survive the image cleaning. Skipping...
--> 23791 event (event ID: 127653) could not survive the image cleaning. Skipping...
--> 23792 event (event ID: 127654) could not survive the image cleaning. Skipping...
23800 events
--> 23808 event (event ID: 127670) could not survive the image cleaning. Skipping...
--> 23809 event (event ID: 127671) could not survive the image cleaning. Skipping...
--> 23815 event (event ID: 127677) could not survive the image cleaning. Skipping...
--> 23826 event (event ID: 127688) could not survive the image cleaning. Skipping...
--> 23827 event (event ID: 127689) could not survive the image cleaning. Skipping...
--> 23829 event (event ID: 127691) could not survive the image cleaning. Skipping...
--> 23835 event (event ID: 127697) could not survive the image cleaning. Skipping...
--> 23844 event (event ID: 127707) could not survive the image cleaning. Skipping...
--> 23850 event (event ID: 127713) could not survive

--> 24260 event (event ID: 128149) could not survive the image cleaning. Skipping...
--> 24265 event (event ID: 128154) could not survive the image cleaning. Skipping...
--> 24277 event (event ID: 128167) could not survive the image cleaning. Skipping...
--> 24278 event (event ID: 128168) could not survive the image cleaning. Skipping...
--> 24296 event (event ID: 128186) could not survive the image cleaning. Skipping...
--> 24299 event (event ID: 128189) could not survive the image cleaning. Skipping...
24300 events
--> 24303 event (event ID: 128193) could not survive the image cleaning. Skipping...
--> 24315 event (event ID: 128206) could not survive the image cleaning. Skipping...
--> 24329 event (event ID: 128222) could not survive the image cleaning. Skipping...
--> 24338 event (event ID: 128231) could not survive the image cleaning. Skipping...
--> 24342 event (event ID: 128238) could not survive the image cleaning. Skipping...
--> 24344 event (event ID: 128240) could not survive

--> 24886 event (event ID: 128819) could not survive the image cleaning. Skipping...
--> 24888 event (event ID: 128821) could not survive the image cleaning. Skipping...
--> 24889 event (event ID: 128822) could not survive the image cleaning. Skipping...
--> 24894 event (event ID: 128828) could not survive the image cleaning. Skipping...
--> 24895 event (event ID: 128829) could not survive the image cleaning. Skipping...
24900 events
--> 24901 event (event ID: 128835) could not survive the image cleaning. Skipping...
--> 24915 event (event ID: 128851) could not survive the image cleaning. Skipping...
--> 24916 event (event ID: 128852) could not survive the image cleaning. Skipping...
--> 24918 event (event ID: 128854) could not survive the image cleaning. Skipping...
--> 24922 event (event ID: 128859) could not survive the image cleaning. Skipping...
--> 24932 event (event ID: 128869) could not survive the image cleaning. Skipping...
--> 24948 event (event ID: 128885) could not survive

--> 25497 event (event ID: 129472) could not survive the image cleaning. Skipping...
25500 events
--> 25500 event (event ID: 129476) could not survive the image cleaning. Skipping...
--> 25516 event (event ID: 129492) could not survive the image cleaning. Skipping...
--> 25519 event (event ID: 129495) could not survive the image cleaning. Skipping...
--> 25520 event (event ID: 129496) could not survive the image cleaning. Skipping...
--> 25529 event (event ID: 129506) could not survive the image cleaning. Skipping...
--> 25536 event (event ID: 129513) could not survive the image cleaning. Skipping...
--> 25545 event (event ID: 129523) could not survive the image cleaning. Skipping...
--> 25546 event (event ID: 129524) could not survive the image cleaning. Skipping...
--> 25547 event (event ID: 129525) could not survive the image cleaning. Skipping...
--> 25551 event (event ID: 129529) could not survive the image cleaning. Skipping...
--> 25569 event (event ID: 129548) could not survive

--> 25989 event (event ID: 130007) could not survive the image cleaning. Skipping...
--> 25992 event (event ID: 130010) could not survive the image cleaning. Skipping...
--> 25994 event (event ID: 130014) could not survive the image cleaning. Skipping...
--> 25996 event (event ID: 130016) could not survive the image cleaning. Skipping...
26000 events
--> 26004 event (event ID: 130025) could not survive the image cleaning. Skipping...
--> 26014 event (event ID: 130036) could not survive the image cleaning. Skipping...
--> 26017 event (event ID: 130039) could not survive the image cleaning. Skipping...
--> 26034 event (event ID: 130058) could not survive the image cleaning. Skipping...
--> 26042 event (event ID: 130066) could not survive the image cleaning. Skipping...
--> 26044 event (event ID: 130068) failed to extract finite timing parameters. Skipping...
--> 26049 event (event ID: 130073) could not survive the image cleaning. Skipping...
--> 26066 event (event ID: 130091) could not s

In [7]:
!magic_calib_to_dl1\
--input-file $magic_input_file_m2\
--output-dir $out_magic_to_dl1\
--config-file $config_file\
--process-run


Input file: /home/mpp/Software/LST_SW_2024/MCP/dl0_dl1_coincidence_stereo/input/MAGIC/M2/20210214_M2_05094483.008_Y_CrabNebula-W0.40+215.root

Is simulation: False

Observation ID: 5094483
Telescope ID: 2

Is stereo trigger: True
Is SUM trigger: False

The following files are found to read drive reports:
/home/mpp/Software/LST_SW_2024/MCP/dl0_dl1_coincidence_stereo/input/MAGIC/M2/20210214_M2_05094483.008_Y_CrabNebula-W0.40+215.root
/home/mpp/Software/LST_SW_2024/MCP/dl0_dl1_coincidence_stereo/input/MAGIC/M2/20210214_M2_05094483.009_Y_CrabNebula-W0.40+215.root

Process run: True

MAGIC image cleaning:
    use_time: True
    use_sum: True
    picture_thresh: 6
    boundary_thresh: 3.5
    max_time_off: 4.5
    max_time_diff: 1.5
    find_hotpixels: True
    pedestal_type: from_extractor_rndm 

Processing the events...
Interpolating cosmic events information from 16 drive reports.
Drive reports available from 2021-02-13 21:54:07.862 to 2021-02-13 21:56:14.006.
0 events
--> 3 event (event

--> 814 event (event ID: 102260) could not survive the image cleaning. Skipping...
--> 822 event (event ID: 102268) could not survive the image cleaning. Skipping...
--> 849 event (event ID: 102298) could not survive the image cleaning. Skipping...
--> 855 event (event ID: 102304) could not survive the image cleaning. Skipping...
--> 884 event (event ID: 102334) could not survive the image cleaning. Skipping...
--> 897 event (event ID: 102347) could not survive the image cleaning. Skipping...
900 events
--> 909 event (event ID: 102360) could not survive the image cleaning. Skipping...
--> 932 event (event ID: 102384) could not survive the image cleaning. Skipping...
--> 940 event (event ID: 102394) could not survive the image cleaning. Skipping...
--> 948 event (event ID: 102402) could not survive the image cleaning. Skipping...
--> 949 event (event ID: 102403) could not survive the image cleaning. Skipping...
--> 977 event (event ID: 102433) could not survive the image cleaning. Skipp

--> 1754 event (event ID: 103255) could not survive the image cleaning. Skipping...
--> 1791 event (event ID: 103293) could not survive the image cleaning. Skipping...
--> 1792 event (event ID: 103294) could not survive the image cleaning. Skipping...
1800 events
--> 1803 event (event ID: 103305) could not survive the image cleaning. Skipping...
--> 1805 event (event ID: 103307) could not survive the image cleaning. Skipping...
--> 1807 event (event ID: 103309) could not survive the image cleaning. Skipping...
--> 1815 event (event ID: 103317) could not survive the image cleaning. Skipping...
--> 1832 event (event ID: 103334) could not survive the image cleaning. Skipping...
--> 1869 event (event ID: 103374) could not survive the image cleaning. Skipping...
--> 1872 event (event ID: 103377) could not survive the image cleaning. Skipping...
--> 1881 event (event ID: 103386) could not survive the image cleaning. Skipping...
--> 1898 event (event ID: 103404) could not survive the image cl

--> 2584 event (event ID: 104123) could not survive the image cleaning. Skipping...
--> 2593 event (event ID: 104132) could not survive the image cleaning. Skipping...
2600 events
--> 2602 event (event ID: 104143) could not survive the image cleaning. Skipping...
--> 2603 event (event ID: 104144) could not survive the image cleaning. Skipping...
--> 2606 event (event ID: 104147) could not survive the image cleaning. Skipping...
--> 2608 event (event ID: 104149) could not survive the image cleaning. Skipping...
--> 2618 event (event ID: 104160) could not survive the image cleaning. Skipping...
--> 2620 event (event ID: 104162) could not survive the image cleaning. Skipping...
--> 2659 event (event ID: 104204) could not survive the image cleaning. Skipping...
--> 2670 event (event ID: 104216) could not survive the image cleaning. Skipping...
--> 2686 event (event ID: 104233) could not survive the image cleaning. Skipping...
--> 2687 event (event ID: 104234) could not survive the image cl

--> 3087 event (event ID: 104715) could not survive the image cleaning. Skipping...
--> 3091 event (event ID: 104720) could not survive the image cleaning. Skipping...
--> 3092 event (event ID: 104722) could not survive the image cleaning. Skipping...
--> 3095 event (event ID: 104729) could not survive the image cleaning. Skipping...
--> 3097 event (event ID: 104731) could not survive the image cleaning. Skipping...
--> 3098 event (event ID: 104732) could not survive the image cleaning. Skipping...
--> 3099 event (event ID: 104733) could not survive the image cleaning. Skipping...
3100 events
--> 3104 event (event ID: 104738) could not survive the image cleaning. Skipping...
--> 3106 event (event ID: 104740) could not survive the image cleaning. Skipping...
--> 3116 event (event ID: 104753) could not survive the image cleaning. Skipping...
--> 3119 event (event ID: 104757) could not survive the image cleaning. Skipping...
--> 3120 event (event ID: 104758) could not survive the image cl

...

--> 22487 event (event ID: 125923) could not survive the image cleaning. Skipping...
--> 22488 event (event ID: 125924) could not survive the image cleaning. Skipping...
22500 events
--> 22513 event (event ID: 125953) could not survive the image cleaning. Skipping...
--> 22520 event (event ID: 125960) could not survive the image cleaning. Skipping...
--> 22522 event (event ID: 125962) could not survive the image cleaning. Skipping...
--> 22524 event (event ID: 125964) could not survive the image cleaning. Skipping...
--> 22536 event (event ID: 125976) could not survive the image cleaning. Skipping...
--> 22542 event (event ID: 125982) could not survive the image cleaning. Skipping...
--> 22551 event (event ID: 125992) could not survive the image cleaning. Skipping...
--> 22556 event (event ID: 125998) could not survive the image cleaning. Skipping...
--> 22565 event (event ID: 126007) could not survive the image cleaning. Skipping...
--> 22571 event (event ID: 126013) could not survive

--> 23320 event (event ID: 126795) could not survive the image cleaning. Skipping...
--> 23323 event (event ID: 126799) could not survive the image cleaning. Skipping...
--> 23348 event (event ID: 126825) could not survive the image cleaning. Skipping...
--> 23350 event (event ID: 126827) could not survive the image cleaning. Skipping...
--> 23365 event (event ID: 126842) could not survive the image cleaning. Skipping...
--> 23389 event (event ID: 126870) could not survive the image cleaning. Skipping...
--> 23394 event (event ID: 126875) could not survive the image cleaning. Skipping...
23400 events
--> 23402 event (event ID: 126885) could not survive the image cleaning. Skipping...
--> 23408 event (event ID: 126891) could not survive the image cleaning. Skipping...
--> 23409 event (event ID: 126892) could not survive the image cleaning. Skipping...
--> 23414 event (event ID: 126897) could not survive the image cleaning. Skipping...
--> 23420 event (event ID: 126903) could not survive

--> 24216 event (event ID: 127747) could not survive the image cleaning. Skipping...
--> 24219 event (event ID: 127750) could not survive the image cleaning. Skipping...
--> 24242 event (event ID: 127775) could not survive the image cleaning. Skipping...
--> 24246 event (event ID: 127779) could not survive the image cleaning. Skipping...
--> 24248 event (event ID: 127781) could not survive the image cleaning. Skipping...
--> 24251 event (event ID: 127784) could not survive the image cleaning. Skipping...
--> 24260 event (event ID: 127795) could not survive the image cleaning. Skipping...
--> 24270 event (event ID: 127805) could not survive the image cleaning. Skipping...
--> 24287 event (event ID: 127823) could not survive the image cleaning. Skipping...
--> 24288 event (event ID: 127824) could not survive the image cleaning. Skipping...
--> 24289 event (event ID: 127825) could not survive the image cleaning. Skipping...
--> 24292 event (event ID: 127828) could not survive the image cl

--> 25096 event (event ID: 128679) failed to extract finite timing parameters. Skipping...
25100 events
--> 25103 event (event ID: 128687) could not survive the image cleaning. Skipping...
--> 25107 event (event ID: 128692) could not survive the image cleaning. Skipping...
--> 25109 event (event ID: 128694) could not survive the image cleaning. Skipping...
--> 25123 event (event ID: 128710) could not survive the image cleaning. Skipping...
--> 25124 event (event ID: 128711) could not survive the image cleaning. Skipping...
--> 25143 event (event ID: 128731) could not survive the image cleaning. Skipping...
--> 25148 event (event ID: 128736) could not survive the image cleaning. Skipping...
--> 25162 event (event ID: 128750) could not survive the image cleaning. Skipping...
--> 25179 event (event ID: 128768) could not survive the image cleaning. Skipping...
--> 25182 event (event ID: 128771) could not survive the image cleaning. Skipping...
--> 25192 event (event ID: 128782) could not s

--> 25899 event (event ID: 129526) could not survive the image cleaning. Skipping...
25900 events
--> 25904 event (event ID: 129531) could not survive the image cleaning. Skipping...
--> 25907 event (event ID: 129534) failed to extract finite timing parameters. Skipping...
--> 25921 event (event ID: 129548) could not survive the image cleaning. Skipping...
--> 25927 event (event ID: 129554) could not survive the image cleaning. Skipping...
--> 25929 event (event ID: 129556) could not survive the image cleaning. Skipping...
--> 25933 event (event ID: 129560) could not survive the image cleaning. Skipping...
--> 25935 event (event ID: 129562) could not survive the image cleaning. Skipping...
--> 25943 event (event ID: 129571) failed to extract finite timing parameters. Skipping...
--> 25952 event (event ID: 129580) could not survive the image cleaning. Skipping...
--> 25980 event (event ID: 129610) could not survive the image cleaning. Skipping...
--> 25981 event (event ID: 129611) could

We will end up with two HDF5 files, one for M1 and one for M2. They contain the Hillas parameters for each event. If there are events where only M1 or M2 survives, they are stored anyway, since they might have triggered LST-1, so that one can still do a stereoscopic analysis with those events.

In [8]:
# if you do not want to wait for the two processes above to finish,
# there are output files already available in IT container or PIC
# just uncomment one of the two lines below.
# if you work locally, the 'output' directory is available in the
# directory of the dataset for this exercises
#out_magic_to_dl1 = "/fefs/aswg/workspace/analysis-school-2024/MCP/dl0_dl1_coincidence_stereo/output/dl0_to_dl1" IT container
#out_magic_to_dl1 = "/pnfs/pic.es/data/magic/Users/analysis-school-2024/MCP/dl0_dl1_coincidence_stereo/output/dl0_to_dl1" PIC
!ls $out_magic_to_dl1

dl1_M1.Run05094483.h5  dl1_M2.Run05094483.h5


In [9]:
import pandas as pd

/tmp/ipykernel_6819/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [10]:
df_magic = pd.read_hdf(f"{out_magic_to_dl1}/dl1_M1.Run05094483.h5", key="events/parameters")

In [11]:
df_magic.head()

,obs_id,event_id,tel_id,pointing_alt,pointing_az,n_pixels,n_islands,tels_with_trigger,time_sec,time_nanosec,...,width,width_uncertainty,psi,intercept,deviation,slope,pixels_width_1,pixels_width_2,intensity_width_1,intensity_width_2
0,5094483,101647,2,1.356704,4.153216,13,1,12,1613253246,863719400,...,0.021325,0.001318,-77.698046,14.893902,0.390143,-0.785664,0.000000,0.000000,0.000000,0.000000
1,5094483,101648,2,1.356704,4.153218,6,1,12,1613253246,874704200,...,0.012891,0.001283,-11.234004,14.318125,0.425586,-5.983845,0.000000,0.000000,0.000000,0.000000
2,5094483,101650,2,1.356703,4.153221,58,3,12,1613253246,889958000,...,0.065763,0.001618,20.960750,16.584101,0.520959,-3.981428,0.004812,0.009625,0.096326,0.188398
3,5094483,101651,2,1.356703,4.153222,15,1,12,1613253246,894446800,...,0.020027,0.001505,-27.275508,14.072912,0.471732,-5.708893,0.000000,0.000000,0.000000,0.000000
4,5094483,101653,2,1.356702,4.153225,179,2,12,1613253246,906102000,...,0.136851,0.001583,-9.291565,14.180575,1.246654,-1.008961,0.011550,0.023099,0.058360,0.129374


Note that `tel_id` is equal to 2 for M1, as configured in the configuration file. All the other parameters comes from the computation of the Hillas and timing parameters.

`tels_with_trigger` is used only for hardware stereo trigger data (for the moment partially supported in MCP).

**NB: the format of the DL1b files generated by `magic_calib_to_dl1` does not follow (for the moment, hopefully will change in the future) the `ctapipe` DL1 format (see https://ctapipe.readthedocs.io/en/latest/user-guide/data_models/dl1.html#dl1-data-model), so you cannot use e.g. the `HDF5EventSource` to read the HDF5 files produced by `magic_calib_to_dl1` (or the `ctapipe` `process` tool).**

### Full analysis scenario

When performing the full analysis, what you would do is to take the first subrun file for each run and each telescope (e.g. `20210214_M1_05094483.001_Y_CrabNebula-W0.40+215.root` and `20210214_M2_05094483.001_Y_CrabNebula-W0.40+215.root` for our example) and run `magic_calib_to_dl1` over them, using the `--process-run` option. Therefore, if you have e.g. 8 runs, you will end up with 16 files (8 for M1 and 8 for M2).

## Coincidence between MAGIC and LST-1 events

Now that we have MAGIC DL1b files, LST-1 enters the game. For the data we are considering, the data were taken with no hardware stereo trigger between MAGIC and LST-1. So, while the telescopes were pointing the same position in the sky, there was no common trigger between the two systems, which took data independently.

For such reason, we can perform only a software trigger, by looking at the timestamps of the events as recorded by MAGIC and LST-1, and looking for coincident events.

To perform the coincidence, one has to take into account that there is a time offset between the events recorded by MAGIC and LST-1 due to the different light path that photons travel to reach the telescopes (plus, a constant offset due to cable lenghts).

Here the important part of the configuration file is:

```
event_coincidence:
    # History of the time offset between LST-1 and MAGIC
    # before June 12 2021: -3.1 us
    # June 13 2021 to Feb 28 2023: -6.5 us
    # March 10 2023 to March 30 2023: -76039.3 us
    # April 13 2023 to August 2023: -25.1 us
    # after Sep 11 2023 : -6.2 us
    timestamp_type_lst: "dragon_time"  # select "dragon_time", "tib_time" or "ucts_time"
    window_half_width: "300 ns"
    pre_offset_search: false
    n_pre_offset_search_events: 100
    time_offset:
        start: "-10 us"
        stop: "0 us"
```

You can see that the offset is not always the same in different periods. Thanks to Seiya (いいね！), there is a time offset history available in the configuration file to help analyzers (to be updated). However, it can happen that in some days the offset is not the one noted above (this is because the LST-1 timing was shifted more).

The event coincidence can be performed in two ways.

### Standard coincidence search

By specifying `pre_offset_search : false` and values for `start` and `stop` for `time_offset`. In that case, the script will scan the number of coincidences between MAGIC and LST-1 as a function of the time offset (from `start` to `stop` in steps of 0.1 us) in a time window defined by `2*window_half_width`. Then the time offset value giving the highest number of coincidences is found (or the mean of the time offsets giving the same highest number of coincidences), let's call it $t_\text{offset,max}$. The final value of the time offset is computed as the weithed average of the time offsets within `4*window_half_width` (i.e. twice the time window) centered on $t_\text{offset,max}$. The weights are the number of coincidences on those time offsets.

<img src="files/coincidence_no_preoffset.png">

As you can guess, this first method assumes that the offset is somewhat known with good precision. If that is not the case, the so-called pre-offset search introduced by Seiya (いいね！) can be used:

### Pre-offset coincidence search

Set `pre_offset_search : true` to use this method.

- extract N (specified by `n_pre_offset_search_events`, default is 100) largest MAGIC/LST events
- compute all possible time combinations (N x N) between extracted MAGIC/LST events
- search the best time offset by counting the number of coincident events using N x N events with an assumption of one of the offset combinations
- when the best time offset is found by the pre-offset search, the final time offset is found from the scan as in the standard coincidence search

Here, use `window_half_width` to tell the script the range over which it will search for coincident events. This is useful when one does not know the time offset. Still, one needs to have an idea of the time offset (even if not precisely). For example, if there is a time offset of seconds, the default window half width is not enough.

### Coincidence hands-on

For this hands-on, I will use the standard search and also the pre-offset, so that one can see how they work.

In [12]:
lst1_input_file = "/home/mpp/Software/LST_SW_2024/MCP/dl0_dl1_coincidence_stereo/input/LST/dl1_LST-1.Run03631.0000.h5"
#lst1_input_file = "/fefs/aswg/workspace/analysis-school-2024/MCP/dl0_dl1_coincidence_stereo/input/LST/dl1_LST-1.Run03631.0000.h5" IT container
#lst1_input_file = "/pnfs/pic.es/data/magic/Users/analysis-school-2024/MCP/dl0_dl1_coincidence_stereo/input/LST/dl1_LST-1.Run03631.0000.h5" PIC

In [13]:
out_coincidence = "./output/coincidence"
!mkdir -p $out_coincidence

In [14]:
config_file_preoffset = "/home/mpp/Software/LST_SW_2024/MCP/dl0_dl1_coincidence_stereo/conf/config_preoffset.yaml"
#config_file_preoffset = "/fefs/aswg/workspace/analysis-school-2024/MCP/dl0_dl1_coincidence_stereo/conf/config_preoffset.yaml" IT container
#config_file_preoffset = "/pnfs/pic.es/data/magic/Users/analysis-school-2024/MCP/dl0_dl1_coincidence_stereo/conf/config_preoffset.yaml" PIC
! cat $config_file_preoffset

mc_tel_ids:
    LST-1: 1
    LST-2: 0
    LST-3: 0
    LST-4: 0
    MAGIC-I: 2
    MAGIC-II: 3


LST:
    image_extractor:
        type: "LocalPeakWindowSum"
        window_shift: 4
        window_width: 8

    increase_nsb:
        use: true
        extra_noise_in_dim_pixels: 1.27
        extra_bias_in_dim_pixels: 0.665
        transition_charge: 8
        extra_noise_in_bright_pixels: 2.08

    increase_psf:
        use: false
        fraction: null

    tailcuts_clean:
        picture_thresh: 8
        boundary_thresh: 4
        keep_isolated_pixels: false
        min_number_picture_neighbors: 2

    time_delta_cleaning:
        use: true
        min_number_neighbors: 1
        time_limit: 2

    dynamic_cleaning:
        use: true
        threshold: 267
        fraction: 0.03

    use_only_main_island: false


MAGIC:
    image_extractor:
        type: "SlidingWindowMaxSum"
        window_width: 5
        apply_integration_correction: false

    charge_correction:
        use: true


In [15]:
!lst1_magic_event_coincidence -h

usage: lst1_magic_event_coincidence [-h] --input-file-lst INPUT_FILE_LST
                                    --input-dir-magic INPUT_DIR_MAGIC
                                    [--output-dir OUTPUT_DIR]
                                    [--config-file CONFIG_FILE]

options:
  -h, --help            show this help message and exit
  --input-file-lst INPUT_FILE_LST, -l INPUT_FILE_LST
                        Path to an input LST DL1 data file
  --input-dir-magic INPUT_DIR_MAGIC, -m INPUT_DIR_MAGIC
                        Path to a directory where input MAGIC DL1 data files
                        are stored
  --output-dir OUTPUT_DIR, -o OUTPUT_DIR
                        Path to a directory where to save an output DL1 data
                        file
  --config-file CONFIG_FILE, -c CONFIG_FILE
                        Path to a configuration file


In [16]:
!lst1_magic_event_coincidence -l $lst1_input_file -m $out_magic_to_dl1 -o $out_coincidence -c $config_file_preoffset


Input LST DL1 data file: /home/mpp/Software/LST_SW_2024/MCP/dl0_dl1_coincidence_stereo/input/LST/dl1_LST-1.Run03631.0000.h5
LST-1: 32936 events

Input MAGIC directory: ./output/dl0_to_dl1

The following DL1 data files are found:
./output/dl0_to_dl1/dl1_M1.Run05094483.h5
./output/dl0_to_dl1/dl1_M2.Run05094483.h5
MAGIC-I: 21787 events
MAGIC-II: 23404 events

LST timestamp type: dragon_time

Coincidence window half width: 300 ns

Pre offset search will be performed.

Pre offset search using large-intensity shower events is ongoing...

Extracting the MAGIC-I events taken when LST observed for pre offset search...
--> 1330 events are found.

Pre offset search finds -3.0 us as a possible offset

Time offsets scan region:
  start: -3.9 us
  stop: -2.1 us

Extracting the MAGIC-I events taken when LST observed...
--> 1330 events are found.

Checking the event coincidence...
time offset: -3.9 us --> 0 events
time offset: -3.8 us --> 0 events
time offset: -3.7 us --> 1 events
time offset: -3.6 u

In [17]:
!lst1_magic_event_coincidence -l $lst1_input_file -m $out_magic_to_dl1 -o $out_coincidence -c $config_file


Input LST DL1 data file: /home/mpp/Software/LST_SW_2024/MCP/dl0_dl1_coincidence_stereo/input/LST/dl1_LST-1.Run03631.0000.h5
LST-1: 32936 events

Input MAGIC directory: ./output/dl0_to_dl1

The following DL1 data files are found:
./output/dl0_to_dl1/dl1_M1.Run05094483.h5
./output/dl0_to_dl1/dl1_M2.Run05094483.h5
MAGIC-I: 21787 events
MAGIC-II: 23404 events

LST timestamp type: dragon_time

Coincidence window half width: 300 ns

offset scan range defined in the config file will be used.

Time offsets scan region:
  start: -10.0 us
  stop: 0.0 us

Extracting the MAGIC-I events taken when LST observed...
--> 1330 events are found.

Checking the event coincidence...
time offset: -10.0 us --> 0 events
time offset: -9.9 us --> 0 events
time offset: -9.8 us --> 1 events
time offset: -9.7 us --> 2 events
time offset: -9.6 us --> 2 events
time offset: -9.5 us --> 2 events
time offset: -9.4 us --> 2 events
time offset: -9.3 us --> 2 events
time offset: -9.2 us --> 2 events
time offset: -9.1 us -

One can see that the two approaches give the same result, great! :D

As you can see from the output, the script will tell you how many events there are for each "event type" i.e. each two telescope combinations and all three telescopes. Therefore, the resulting HDF5 file is a table with the events DL1b information (either MAGIC only, or MAGIC+LST-1).

In [18]:
df = pd.read_hdf(f"{out_coincidence}/dl1_MAGIC_LST-1.Run03631.0000.h5", key="events/parameters")
df.head()

,obs_id,event_id,tel_id,obs_id_magic,event_id_magic,obs_id_lst,event_id_lst,intensity,x,y,...,pixels_width_1,pixels_width_2,n_pixels,n_islands,pointing_alt,pointing_az,timestamp,time_diff,multiplicity,combo_type
0,3631,16,1,5094483,102313,3631.0,16.0,304.667151,0.057675,-0.414797,...,0.000000,0.000000,14,2,1.356512,4.153884,1.613253e+09,0.000120,2,2
1,3631,16,3,5094483,102313,3631.0,16.0,64.476562,0.004862,-0.107461,...,0.000000,0.000000,7,1,1.356456,4.153876,1.613253e+09,0.000923,2,2
2,3631,24,1,5094483,102314,3631.0,24.0,1155.765821,0.401519,0.354602,...,0.000000,0.000000,75,1,1.356512,4.153885,1.613253e+09,0.000308,3,1
3,3631,24,2,5094483,102314,3631.0,24.0,63.041406,-0.027460,0.304506,...,0.000000,0.000000,9,1,1.356521,4.153870,1.613253e+09,0.006442,3,1
4,3631,24,3,5094483,102314,3631.0,24.0,613.492188,0.120570,0.322906,...,0.005775,0.014437,71,2,1.356456,4.153876,1.613253e+09,0.002970,3,1


The `combo_type` column tells which telescope combination corresponds to the given event:

* 0 = LST-1+M1
* 1 = LST-1+M1+M2
* 2 = LST-1+M2
* 3 = M1+M2

### Full analysis scenario

When performing the full analysis, what you would do is to put all the MAGIC DL1b files from a given night and source in a directory. Then you would run `lst1_magic_event_coincidence` for each LST-1 DL1 file i.e. each subrun. This will give you a MAGIC+LST-1 DL1 file for each LST-1 subrun (assuming each has coincident events).

At this point, one could use the auxiliary script `merge_hdf_files` to merge the DL1 files created in this step, so that there is just one DL1 file for each run. You would call it like this:

```
merge_hdf_files -i <directory_with_dl1_files> -o <output_directory> --run-wise
```

NB: you can use this also to merge together the MAGIC DL1 files for M1 and M2.

## Stereo reconstruction

The final step for this tutorial is stereo event reconstruction. For this, we use the `HillasReconstructor` class from `ctapipe`. From this, we get for each event the reconstrucyed direction of the primary particle in local coordinates (az, alt), core position, height of the shower maximum. We also add the impact parameter (closest distance between a shower axis and a telescope position).

We use the `lst1_magic_stereo_reco` script.

In [19]:
out_stereo = "./output/dl1_stereo"
!mkdir $out_stereo

mkdir: cannot create directory ‘./output/dl1_stereo’: File exists


In [20]:
dl1_input = f"{out_coincidence}/dl1_MAGIC_LST-1.Run03631.0000.h5"

From the configuration file, the relevant part is the following:

```
stereo_reco:
    quality_cuts: "(intensity > 50) & (width > 0)"
    theta_uplim: "6 arcmin"
```

i.e. some quality cuts will be applied to the events. `theta_uplim` specifies the maximum angular separation between telescope pointings to allow stereo reconstruction.

In [21]:
!lst1_magic_stereo_reco -h

usage: lst1_magic_stereo_reco [-h] --input-file INPUT_FILE
                              [--output-dir OUTPUT_DIR]
                              [--config-file CONFIG_FILE] [--magic-only]

options:
  -h, --help            show this help message and exit
  --input-file INPUT_FILE, -i INPUT_FILE
                        Path to an input DL1 data file
  --output-dir OUTPUT_DIR, -o OUTPUT_DIR
                        Path to a directory where to save an output DL1-stereo
                        data file
  --config-file CONFIG_FILE, -c CONFIG_FILE
                        Path to a configuration file
  --magic-only          Reconstruct the stereo parameters using only MAGIC
                        events


In [22]:
!lst1_magic_stereo_reco -i $dl1_input -o $out_stereo -c $config_file


Input file: ./output/coincidence/dl1_MAGIC_LST-1.Run03631.0000.h5

Is simulation: False

Telescope positions:
    1: <Quantity  -8.09 77.13  0.78  m>
    2: <Quantity   39.3  -62.55  -0.97  m>
    3: <Quantity  -31.21 -14.57   0.2   m> 

MAGIC-only analysis: False

Quality cuts: (intensity > 50) & (width > 0)

In total 1214 stereo events are found:
    LST-1_MAGIC-I (type 0): 56 events (4.6%)
    LST-1_MAGIC-I_MAGIC-II (type 1): 707 events (58.2%)
    LST-1_MAGIC-II (type 2): 158 events (13.0%)
    MAGIC-I_MAGIC-II (type 3): 293 events (24.1%) 

Checking the angular distances of the LST and MAGIC pointing directions...
--> All the events were taken with smaller angular distances than the limit 6.0 arcmin.

Reconstructing the stereo parameters...
0 events
100 events
200 events
300 events
400 events
--> event 435 (event ID 25885) failed to reconstruct valid stereo parameters, maybe due to the images of zero width. Skipping...
500 events
600 events
700 events
--> event 777 (event ID 4482

Note from the output:

```
--> All the events were taken with smaller angular distances than the limit 6.0 arcmin.
```

The poitings directions between the telescopes cannot be too different. This is a general problem for joint analysis, if the pointings are too far apart, it makes difficult (or impossible) to find some common regions of the sky to get the OFF events from.

### Full analysis scenario

In the full analysis scenario, instead of the single DL1 subrun file, you would run the script over the merged DL1 run file (see above).